In [1]:
import sys

sys.path.append("../src")

In [2]:
import os
import time
import threading

import tensorflow as tf
import numpy as np
from grpc.beta import implementations

from apis import prediction_service_pb2
from apis import predict_pb2

/home/ubuntu/.pyenv/versions/3.6.2/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Run request and show result

In [4]:
host = "192.168.1.113"
port = 9000

image_path = "/home/ubuntu/Downloads/20171204_161643.jpg"
with open(image_path, "rb") as f:
    data = f.read()

In [5]:
channel = implementations.insecure_channel(host, int(port))
stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)

In [6]:
request = predict_pb2.PredictRequest()
request.model_spec.name = "recycle"
request.model_spec.signature_name = "scores"
request.inputs["image"].CopyFrom(tf.contrib.util.make_tensor_proto([data], dtype=tf.string))

In [7]:
result = stub.Predict(request, 10.0)
result

outputs {
  key: "class_id"
  value {
    dtype: DT_INT64
    tensor_shape {
      dim {
        size: 1
      }
    }
    int64_val: 17
  }
}
outputs {
  key: "prob"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
    }
    float_val: 0.8333612084388733
  }
}

In [15]:
logits = tf.contrib.util.make_ndarray(result.outputs["logits"])
logits

array([[ -6.01338387, -13.18535233,  -8.02176952,  -6.75897646,
         -9.61907005,  -8.93222427, -10.33878422, -13.82873154,
         -6.57908678, -11.15718937, -15.23946953, -10.56701279,
         -9.20000935,  -9.91709995,  -9.73126125, -12.0344038 ,
         -5.10550308, -12.97931862,   1.36900067,  11.26507664,
         -8.05778027,  -8.55801105, -16.73856354, -12.15321445,
        -15.29555225,  -7.55768776,  -1.24826717,  -8.15581703,
         -8.61640263,  -6.9349575 ]], dtype=float32)

In [18]:
np.argmax(logits)

19

# Test serving speed

In [5]:
def send_request(num_runs):
    channel = implementations.insecure_channel(host, int(port))
    stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)
    
    request = predict_pb2.PredictRequest()
    request.model_spec.name = "recycle"
    request.model_spec.signature_name = "scores"
    request.inputs["image"].CopyFrom(tf.contrib.util.make_tensor_proto([data], dtype=tf.string))
    
    for _ in range(num_runs):
        result = stub.Predict(request, 10.0)

In [10]:
num_threads = 16
num_runs = 100
thr = []
for _ in range(num_threads):
    t = threading.Thread(target=send_request, args=(num_runs,))
    thr.append(t)
    
start_time = time.time()
[t.start() for t in thr]
[t.join() for t in thr]
elapsed = time.time() - start_time
print("Elapsed: {}, time per request: {}".format(elapsed, elapsed / num_runs / num_threads))

Elapsed: 17.511889219284058, time per request: 0.010944930762052536


# Check accuracy of running model

In [9]:
channel = implementations.insecure_channel(host, int(port))
stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)

request = predict_pb2.PredictRequest()
request.model_spec.name = "recycle"
request.model_spec.signature_name = "scores"

In [18]:
folder = "/home/ubuntu/github/ai/recycle/data/raw/merged_v2/split/val/non_07_Batteries/"
class_ids = []
for dirpath, dirnames, filenames in os.walk(folder):
    for filename in sorted(filenames):
        image_path = os.path.join(dirpath, filename)
        with open(image_path, "rb") as f:
            data = f.read()
            
        request.inputs["image"].CopyFrom(tf.contrib.util.make_tensor_proto([data], dtype=tf.string))
        result = stub.Predict(request, 10.0)
        class_id = tf.contrib.util.make_ndarray(result.outputs["class_id"])[0]
        class_ids.append(class_id)
        print("Done {}, class_id {}".format(filename, class_id))
        
print(np.mean((np.array(class_ids) == 6).astype(np.float32)))

Done rechargeable_battery_000011.jpg, class_id 6
Done rechargeable_battery_000012.png, class_id 6
Done rechargeable_battery_000072.jpg, class_id 6
Done rechargeable_battery_000085.jpg, class_id 6
Done rechargeable_battery_000114.jpeg, class_id 15
Done rechargeable_battery_000126.jpg, class_id 6
Done rechargeable_battery_000149.JPG, class_id 6
Done rechargeable_battery_000170.jpg, class_id 6
Done rechargeable_battery_000173.jpg, class_id 6
Done rechargeable_battery_000178.jpg, class_id 6
Done rechargeable_battery_000181.png, class_id 6
Done rechargeable_battery_000192.jpg, class_id 6
Done used_battery_000008.png, class_id 6
Done used_battery_000016.jpg, class_id 6
Done used_battery_000052.jpg, class_id 6
Done used_battery_000062.jpg, class_id 6
Done used_battery_000113.jpg, class_id 24
Done used_battery_000120.jpg, class_id 6
Done used_battery_000133.JPG, class_id 6
Done used_battery_000165.jpg, class_id 6
Done used_phone_battery_000030.jpg, class_id 6
Done used_phone_battery_000129.jpg